In [166]:
from riotwatcher import RiotWatcher
import pandas as pd
from converters.data2frames import game_to_dataframe as g2df
from converters.data2frames import game_teams_to_dataframe as gt2df
from converters.data2files import read_json
from config.constants import API_KEY, SLO_GAMES_DIR, SLO_DATASETS_DIR
import numpy as np


In [171]:
df = pd.read_csv('{}{}'.format(SLO_DATASETS_DIR, 'dataset_test.csv'), index_col=0)

In [173]:
import datetime
from time import strftime

In [175]:
df.gameCreation = df.gameCreation.apply(lambda x: datetime.datetime.fromtimestamp(x / 1e3).strftime('%Y-%m-%d %H:%M:%S'))

In [179]:
def get_readable_timestamp(seconds):
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)

    return "{h}:{m}:{s}".format(h=int(h), m=int(m), s=int(s))

In [180]:
df.gameDuration.apply(get_readable_timestamp)

0      0:36:29
1      0:36:29
2      0:36:29
3      0:36:29
4      0:36:29
5      0:36:29
6      0:36:29
7      0:36:29
8      0:36:29
9      0:36:29
10     0:22:29
11     0:22:29
12     0:22:29
13     0:22:29
14     0:22:29
15     0:22:29
16     0:22:29
17     0:22:29
18     0:22:29
19     0:22:29
20      0:31:6
21      0:31:6
22      0:31:6
23      0:31:6
24      0:31:6
25      0:31:6
26      0:31:6
27      0:31:6
28      0:31:6
29      0:31:6
        ...   
290    0:53:12
291    0:53:12
292    0:53:12
293    0:53:12
294    0:53:12
295    0:53:12
296    0:53:12
297    0:53:12
298    0:53:12
299    0:53:12
300     0:36:6
301     0:36:6
302     0:36:6
303     0:36:6
304     0:36:6
305     0:36:6
306     0:36:6
307     0:36:6
308     0:36:6
309     0:36:6
310    0:31:50
311    0:31:50
312    0:31:50
313    0:31:50
314    0:31:50
315    0:31:50
316    0:31:50
317    0:31:50
318    0:31:50
319    0:31:50
Name: gameDuration, Length: 320, dtype: object